In [37]:
import pandas as pd
import numpy as np
from utils import zebra
import re
solver_data = pd.read_excel("2021_data.xlsx", sheet_name='Solver Data')
partner_data =  pd.read_excel("2021_data.xlsx", sheet_name='Partner Data')
solver_data = solver_data.rename(columns={'Solution Name': 'Org'})
partner_data = partner_data.rename(columns={'Organization Name': 'Org',
                                            'Challenge':'Challenge Preference',
                                            'Stage': 'Solution Preference: Organization Stage',
                                            'Expertise': 'Partnership Preference: Non-Financial',
                                           'Solution Preference: Geographies': 'Geo Interest'})


partner_data = partner_data.replace(np.nan, "Noval")
solver_data  = solver_data.replace(np.nan, "Noval")
solver_data_copy = solver_data.copy()

In [38]:
partner_data.columns

Index(['Source', 'Org', 'First Name', 'Last Name', 'Geo Interest',
       'Challenge Preference', 'Solution Preference: Organization Stage',
       'Solution Preference: Organization Type', 'Funding Preference 1',
       'Funding Preference 2', 'Funding Preference 3',
       'Partnership Preference: Non-Financial 1',
       'Partnership Preference: Non-Financial 2',
       'Partnership Preference: Non-Financial 3',
       'Partnership Preference: Non-Financial 4',
       'Partnership Preference: Non-Financial 5',
       'Partnership Preference: Non-Financial 6',
       'Partnership Preference: Non-Financial 7',
       'Partnership Preference: Non-Financial 8',
       'Solution Preference: Technologies',
       'Solution Preferences: Partner Expertise',
       'Partnership Preference: Non-Financial', 'Title',
       'Contact Owner Alias'],
      dtype='object')

In [3]:
  
def split_collect(df_cols, delimiter=','):
    """
        Split each value in a cell based on a delimiter 
        and return a list of unique options 
        
    """
    opts = df_cols.apply(lambda x :  x.split(delimiter)   ).to_list()
    flatten_opts = [x.strip() for y in opts for x in y ]
    opts = pd.DataFrame(data=flatten_opts, columns=['options'])
    opts = opts['options'].value_counts().index.to_list()
    return opts


def expand_col(df_col, delimiter=',',col_name='new_col'): 
    """
    Take in a pandas series whose elements are
    a string. Split each cell of the series with
    a delimiter which is used togenerate an N column dataframe. 
    N is the longest list amongst the cells of df_col after
    they have been split
    
    """
#     df_col = df_col.apply(lambda x : x.str.split(delimiter)).to_list()
    df_col = df_col.str.split(delimiter).to_list()
    new_df = pd.DataFrame(data=df_col)
    ncols = len(new_df.columns)
    new_names = []
    for x in range(1, ncols+1): 
        new_name ="".join((col_name,'_', str(x)))
        new_df = new_df.rename(columns={x-1:new_name})
    return new_df


def match_single_to_multi(single_df, multi_df, single_match_on='None'): 
    """
    Generate a pivot table between a df which has a single of choices 
    and a df with multiple columns of choices
    
    """
    melted_df = pd.melt(multi_df,id_vars='Org')
    melted_df = melted_df.drop(columns='variable')
    melted_df = melted_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    single_df = single_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    matched_df = pd.merge(melted_df, single_df, how='outer', left_on='value', right_on=single_match_on)
    matched_df = matched_df.replace(np.nan, 'Noval')
    matched_df['value'] = matched_df['value'].apply(lambda x : 0 if x == None else 1)
    pivot_table = pd.pivot_table(matched_df, index='Org_x', columns=['Org_y'], values='value', dropna=False,  aggfunc=np.sum)

    return pivot_table


def match_multi(df1, df2):
    """
    Match a feature with multiple options to another option with multiple options
    """
    
    melted_df1 = pd.melt(df1,id_vars='Org').fillna('Noval')
    melted_df2 = pd.melt(df2, id_vars='Org').fillna('Noval')
    melted_df1 = melted_df1.drop(columns='variable')
    melted_df2 = melted_df2.drop(columns='variable')
    melted_df1 = melted_df1.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    melted_df2 = melted_df2.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    matched_df = pd.merge(melted_df1, melted_df2, how='outer', left_on='value', right_on='value')
    matched_df = matched_df.replace(np.nan, 'Noval')
    matched_df['value'] = matched_df['value'].apply(lambda x : 0 if  x == 'Noval' else 1)
    pivot_table = pd.pivot_table(matched_df, index='Org_x', columns=['Org_y'], values='value',aggfunc=np.sum)
    return pivot_table
                              

## Challenge column cleaning and matching


In [4]:
partner_challenge  = partner_data['Challenge Preference']
partner_data['Challenge Preference'] = partner_data['Challenge Preference'].apply(lambda x: x.strip().replace(';',',') if ';' in x else x.strip() )
partner_challenge_cols = expand_col(partner_data['Challenge Preference'], col_name ='Challenge')
partner_challenge_opts = split_collect(partner_data['Challenge Preference'])


In [5]:
solver_challenge_opts = split_collect(solver_data['Challenge'])
set(solver_challenge_opts).issubset(set(partner_challenge_opts))

True

## Add challenge columns to partner data sheet

In [6]:
partner_data_updated = partner_data.copy()
partner_data_updated = partner_data_updated.drop(columns='Challenge Preference')
partner_data_updated = pd.concat([partner_data_updated, partner_challenge_cols], axis=1)

partner_data_updated

,Source,Org,First Name,Last Name,Geo Interest,Solution Preference: Organization Stage,Solution Preference: Organization Type,Funding Preference 1,Funding Preference 2,Funding Preference 3,...,Challenge_5,Challenge_6,Challenge_7,Challenge_8,Challenge_9,Challenge_10,Challenge_11,Challenge_12,Challenge_13,Challenge_14
0,SF Export,Access Afya,Melissa,Menke,Sub-Saharan Africa; South Asia,Noval,Noval,Noval,Noval,Noval,...,None,None,None,None,None,None,None,None,None,None
1,SF Export,Amazon,Nicola,Poirier,North America,Concept; Prototype; Pilot; Growth; Scale,For-Profit; Nonprofit; Not Registered as Any O...,Noval,Noval,Noval,...,None,None,None,None,None,None,None,None,None,None
2,SF Export,American Family Insurance Institute for Corpor...,John,McIntyre,North America,Concept; Prototype; Pilot; Growth; Scale,For-Profit,Pre-committed prize funding,Equity funding,Noval,...,Work of the Future,Teachers & Educators,Coastal Communities,Women & Technology,Brain Health,Youth Skills & the Workforce of the Future,Sustainable Urban Communities,Inclusive Innovation,None,None
3,SF Export,American Student Assistance (ASA),Annabel,Cellini,North America,Concept; Prototype; Pilot; Growth; Scale,For-Profit; Nonprofit; Hybrid of For-Profit & ...,Grant funding,Equity funding,Noval,...,None,None,None,None,None,None,None,None,None,None
4,SF Export,Americares,Sandy,Dickson,Latin America and the Caribbean; Middle East a...,Pilot; Growth; Scale,For-Profit; Nonprofit; Not Registered as Any O...,Grant funding,Equity funding,Noval,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,SF Export,Wiley Education Services,Andrew,Tein,East Asia and Pacific; South Asia; North America,Concept; Prototype; Pilot; Growth; Scale,For-Profit; Nonprofit; Not Registered as Any O...,Noval,Noval,Noval,...,None,None,None,None,None,None,None,None,None,None
112,SF Export,Women's WorldWide Web (W4),Lindsey,Nefesh-Clarke,Sub-Saharan Africa; East Asia and Pacific,Concept; Prototype; Pilot,Nonprofit; Hybrid of For-Profit & Nonprofit,Pre-committed prize funding,Grant funding,Noval,...,Youth Skills & the Workforce of the Future,Inclusive Innovation,None,None,None,None,None,None,None,None
113,SF Export,X Prize Foundation,Amir,Banifatemi,Latin America and the Caribbean; Europe and Ce...,Growth,Noval,Noval,Noval,Noval,...,None,None,None,None,None,None,None,None,None,None
114,SF Export,"X, The Moonshot Factory",Erica,Bliss,Middle East and North Africa; Sub-Saharan Afri...,Noval,Noval,Noval,Noval,Noval,...,None,None,None,None,None,None,None,None,None,None


## Geo Interests


In [7]:
partner_data_updated['Geo Interest'] = partner_data_updated['Geo Interest'].apply(lambda x: x.strip().replace(';',',') if ';' in x else x.strip() )
partner_geo_opts = split_collect(partner_data_updated['Geo Interest'])
partner_geo_opts

['North America',
 'Sub-Saharan Africa',
 'East Asia and Pacific',
 'Middle East and North Africa',
 'Europe and Central Asia',
 'Latin America and the Caribbean',
 'South Asia']

In [8]:
solver_geo_opts = solver_data[['Geo 1', 'Geo 2', 'Geo 3']].apply(lambda x : ",".join(x)   ,axis=1)
solver_geo_opts = split_collect(solver_geo_opts)
solver_geo_opts

['Noval',
 'Sub-Saharan Africa',
 'South Asia',
 'North America',
 'Latin America and the Caribbean',
 'Middle East and North Africa',
 'East Asia and Pacific',
 'Europe and Central Asia']

In [9]:
set(partner_geo_opts).issubset(set(solver_geo_opts))

True

### Add geo columns to partner datasheet

In [10]:
partner_geo_cols = partner_data_updated['Geo Interest']
partner_geo_cols = expand_col(partner_geo_cols, col_name='geo')

# remove partner geo column and add individual geo columns
partner_data_updated = pd.concat([partner_data_updated, partner_geo_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Geo Interest'])


## Key needs

In [11]:
# Partnership preferences non financal from partner data

pref_name = 'Partnership Preference'
prefs_columns = [x  for x in partner_data_updated.columns if pref_name in x]
partner_prefs_opts = partner_data_updated[prefs_columns]
partner_prefs_opts = partner_prefs_opts.applymap(lambda x: x.strip())
partner_prefs_opts = split_collect(partner_prefs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
partner_prefs_opts

['Noval',
 'Introductions to network',
 'Business model (e.g. product-market fit, strategy & development)',
 'Mentorship',
 'Product / Service Distribution (e.g. expanding client base)',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Pro-bono services (i.e. legal services, business strategy services, etc.)',
 'Incubation services',
 'In-kind resources (i.e. office space, software licenses, products, etc.)',
 'Other (explain below)',
 'Legal or Regulatory Matters']

In [12]:
needs_name = 'Key Need'
needs_columns = [x  for x in solver_data.columns if needs_name in x]
solver_needs_opts = solver_data[needs_columns]
solver_needs_opts = solver_needs_opts.applymap(lambda x: x.strip())
solver_needs_opts = split_collect(solver_needs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
solver_needs_opts

['Noval',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Business model (e.g. product-market fit, strategy & development)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Legal or Regulatory Matters',
 'Other (explain below)',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (i.e. sourcing talent, board development, etc.)']

In [13]:
# Check if all the solver need options are in partner preferences
set(solver_needs_opts).difference(set(partner_prefs_opts))

{'Human Capital (i.e. sourcing talent, board development, etc.)'}

In [14]:
# Check the opposite of the above
set(partner_prefs_opts).difference(set(solver_needs_opts))

{'In-kind resources (i.e. office space, software licenses, products, etc.)',
 'Incubation services',
 'Introductions to network',
 'Mentorship',
 'Pro-bono services (i.e. legal services, business strategy services, etc.)'}

In [15]:
# Correcting Human Captial in partner data
wrong_str1 = 'Human Capital (e.g. sourcing talent, board development, etc.)'
correct_str1 = 'Human Capital (i.e. sourcing talent, board development, etc.)'
partner_data_updated = partner_data_updated.applymap(lambda x: x.strip() if isinstance(x, str) else x)

for pref in prefs_columns:
    partner_data_updated[pref] = partner_data_updated[pref].apply(lambda x: x.strip().replace(wrong_str1, correct_str1) if x == wrong_str1 else x )



In [16]:
# Correcting Capital in solver data
wrong_str1 = 'Human Capital (e.g. sourcing talent, board development, etc.)'
correct_str2 = 'Human Capital (i.e. sourcing talent, board development, etc.)'
solver_data_updated = solver_data.copy()
solver_data_updated = solver_data_updated.applymap(lambda x: x.strip() if isinstance(x, str) else x)

for pref in needs_columns:
    solver_data_updated[pref] = solver_data_updated[pref].apply(lambda x: x.strip().replace(wrong_str1, correct_str1) if x == wrong_str1 else x )
    



In [17]:
# Partnership preferences non financal from partner data

pref_name = 'Partnership Preference'
prefs_columns = [x  for x in partner_data_updated.columns if pref_name in x]
partner_prefs_opts = partner_data_updated[prefs_columns]
partner_prefs_opts = partner_prefs_opts.applymap(lambda x: x.strip())
partner_prefs_opts = split_collect(partner_prefs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
partner_prefs_opts


['Noval',
 'Introductions to network',
 'Business model (e.g. product-market fit, strategy & development)',
 'Mentorship',
 'Product / Service Distribution (e.g. expanding client base)',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (i.e. sourcing talent, board development, etc.)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Pro-bono services (i.e. legal services, business strategy services, etc.)',
 'Incubation services',
 'In-kind resources (i.e. office space, software licenses, products, etc.)',
 'Other (explain below)',
 'Legal or Regulatory Matters']

## Stage Match

In [18]:
partner_data_updated['Solution Preference: Organization Stage'] = partner_data_updated['Solution Preference: Organization Stage'].apply(lambda x : x.replace(';',',' if ';' in x else x))
partner_stage = partner_data_updated['Solution Preference: Organization Stage']
partner_stage_opts = split_collect(partner_stage)

In [19]:
solver_stage_opts = split_collect(solver_data_updated['Stage'])
set(solver_stage_opts).difference(set(partner_stage_opts))

set()

### Add seprated stage columns to partner data sheet


In [20]:
partner_stage_cols = expand_col(partner_stage, col_name='Stage')
partner_data_updated = pd.concat([partner_data_updated, partner_stage_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Solution Preference: Organization Stage'])


## Tech Match

In [21]:
partner_tech_opts = split_collect(partner_data_updated['Solution Preference: Technologies'], delimiter=";")
tech_name = 'Tech'
tech_cols = [x for x in solver_data_updated.columns if  tech_name in x]
tech_cols.append('Org')
solver_tech_cols = solver_data_updated[tech_cols]
solver_tech_opts = split_collect(solver_tech_cols.apply(lambda x : ";".join(x), axis=1), delimiter=";")
set(partner_tech_opts).difference(solver_tech_opts)


{'Biomimicry', 'Blockchain', 'Virtual Reality/Augmented Reality'}

In [22]:
partner_tech_cols = expand_col(partner_data_updated['Solution Preference: Technologies'], col_name='tech')
partner_data_updated = pd.concat([partner_data_updated, partner_tech_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Solution Preference: Technologies'])

# All matching sheets calculated here
##  Challenge Matching sheet


In [23]:
partner_data_updated = partner_data_updated.replace('None', 'Noval' ,regex=True)
solver_data_updated = solver_data_updated.replace('None', 'Noval', regex=True)
partner_data_updated = partner_data_updated.replace(np.nan, "Noval")
solver_data_updated  = solver_data_updated.replace(np.nan, "Noval")
partner_data_updated.columns

Index(['Source', 'Org', 'First Name', 'Last Name',
       'Solution Preference: Organization Type', 'Funding Preference 1',
       'Funding Preference 2', 'Funding Preference 3',
       'Partnership Preference: Non-Financial 1',
       'Partnership Preference: Non-Financial 2',
       'Partnership Preference: Non-Financial 3',
       'Partnership Preference: Non-Financial 4',
       'Partnership Preference: Non-Financial 5',
       'Partnership Preference: Non-Financial 6',
       'Partnership Preference: Non-Financial 7',
       'Partnership Preference: Non-Financial 8',
       'Solution Preferences: Partner Expertise',
       'Partnership Preference: Non-Financial', 'Title', 'Contact Owner Alias',
       'Challenge_1', 'Challenge_2', 'Challenge_3', 'Challenge_4',
       'Challenge_5', 'Challenge_6', 'Challenge_7', 'Challenge_8',
       'Challenge_9', 'Challenge_10', 'Challenge_11', 'Challenge_12',
       'Challenge_13', 'Challenge_14', 'geo_1', 'geo_2', 'geo_3', 'geo_4',
       'geo_

In [24]:
chname = 'Challenge'
challenge_cols = [x for x in partner_data_updated.columns if chname in x ]
partner_challenge_cols = partner_data_updated[challenge_cols]

In [25]:

solver_challenge_cols = solver_data_updated[['Org', 'Challenge']]
partner_challenge_cols['Org'] = partner_data_updated['Org']
challenge_match = match_single_to_multi(solver_challenge_cols, partner_challenge_cols, single_match_on='Challenge')

if 'Noval' in challenge_match.columns: 
    challenge_match = challenge_match.drop(columns=['Noval'])
challenge_match.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(112, 36)

## Geo Matching sheet


In [26]:
geo_cols = [x for x in partner_data_updated.columns if 'geo' in x]
geo_cols.append('Org')
partner_geo_cols = partner_data_updated[geo_cols]

solver_geo_cols = solver_data_updated[['Org', 'Geo 1', 'Geo 2', 'Geo 3']]
geo_match = match_multi(partner_geo_cols,solver_geo_cols)
if 'Noval' in geo_match.columns: 
    geo_match = geo_match.drop(columns=['Noval'])
geo_match.shape

(112, 36)

In [27]:
geo_match_index = geo_match.index.tolist()

## Needs Matching sheet

In [28]:
needs_name ='Key Need'
pref_name = 'Partnership Preference'
prefs_columns = [x  for x in partner_data_updated.columns if pref_name in x]
needs_columns = [x  for x in solver_data_updated.columns if needs_name in x]
prefs_columns.append('Org')
needs_columns.append('Org')
partner_prefs_cols = partner_data_updated[prefs_columns]
solver_needs_cols = solver_data_updated[needs_columns]
needs_match = match_multi(partner_prefs_cols, solver_needs_cols)
if 'Noval' in needs_match.columns: 
    needs_match = needs_match.drop(columns=['Noval'])
needs_match.shape


(112, 36)

In [29]:
needs_match_index = needs_match.index.tolist()

In [30]:
set(needs_match_index).difference(set(geo_match_index))

set()

## Stage Matching sheet

In [31]:
stage_name = 'Stage'
stage_columns = [x  for x in partner_data_updated.columns if stage_name in x]
stage_columns.append('Org')
partner_stage_cols = partner_data_updated[stage_columns]
solver_stage_cols = solver_data_updated[['Stage', 'Org']]
stage_match = match_single_to_multi(solver_stage_cols, partner_stage_cols, single_match_on='Stage')

if 'Noval' in stage_match.columns: 
    stage_match = stage_match.drop(columns=['Noval'])
stage_match.shape

(112, 36)

In [32]:
set(stage_match.columns.to_list()).difference(set(needs_match.columns.to_list()))

set()

## Tech Matching Sheet

In [33]:
tech_name = 'tech'
tech_cols = [x for x in partner_data_updated.columns if  tech_name in x]
tech_cols.append('Org')
partner_tech_cols = partner_data_updated[tech_cols]

In [34]:
partner_tech_cols['Org'] = partner_data_updated['Org']
tech_match = match_multi(partner_tech_cols, solver_tech_cols)
if 'Noval' in tech_match.columns:
    tech_match = tech_match.drop(columns=['Noval'])
if 'Noval' in tech_match.index:
    tech_match = tech_match.drop(index='Noval', axis=0) 
tech_match.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(112, 36)

In [35]:

partner_data_updated.to_excel("partner_data_2021.xlsx")

In [36]:
solver_data_updated.to_excel("solver_data_2021.xlsx")

In [39]:
partner_data_updated

,Source,Org,First Name,Last Name,Solution Preference: Organization Type,Funding Preference 1,Funding Preference 2,Funding Preference 3,Partnership Preference: Non-Financial 1,Partnership Preference: Non-Financial 2,...,geo_4,geo_5,geo_6,geo_7,Stage_1,Stage_2,Stage_3,Stage_4,Stage_5,tech_1
0,SF Export,Access Afya,Melissa,Menke,Noval,Noval,Noval,Noval,Noval,Noval,...,Noval,Noval,Noval,Noval,Noval,Noval,Noval,Noval,Noval,Behavioral Technology; Big Data; Software and ...
1,SF Export,Amazon,Nicola,Poirier,For-Profit; Nonprofit; Not Registered as Any O...,Noval,Noval,Noval,"Business model (e.g. product-market fit, strat...",Financial (e.g. improving accounting practices...,...,Noval,Noval,Noval,Noval,Concept,Prototype,Pilot,Growth,Scale,Ancestral Technology & Practices; Artificial I...
2,SF Export,American Family Insurance Institute for Corpor...,John,McIntyre,For-Profit,Pre-committed prize funding,Equity funding,Noval,Noval,Noval,...,Noval,Noval,Noval,Noval,Concept,Prototype,Pilot,Growth,Scale,Noval
3,SF Export,American Student Assistance (ASA),Annabel,Cellini,For-Profit; Nonprofit; Hybrid of For-Profit & ...,Grant funding,Equity funding,Noval,"Business model (e.g. product-market fit, strat...",Monitoring & Evaluation (e.g. collecting/using...,...,Noval,Noval,Noval,Noval,Concept,Prototype,Pilot,Growth,Scale,Noval
4,SF Export,Americares,Sandy,Dickson,For-Profit; Nonprofit; Not Registered as Any O...,Grant funding,Equity funding,Noval,"Business model (e.g. product-market fit, strat...",Monitoring & Evaluation (e.g. collecting/using...,...,East Asia and Pacific,Noval,Noval,Noval,Pilot,Growth,Scale,Noval,Noval,Noval
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,SF Export,Wiley Education Services,Andrew,Tein,For-Profit; Nonprofit; Not Registered as Any O...,Noval,Noval,Noval,"Business model (e.g. product-market fit, strat...",Noval,...,Noval,Noval,Noval,Noval,Concept,Prototype,Pilot,Growth,Scale,Ancestral Technology & Practices; Artificial I...
112,SF Export,Women's WorldWide Web (W4),Lindsey,Nefesh-Clarke,Nonprofit; Hybrid of For-Profit & Nonprofit,Pre-committed prize funding,Grant funding,Noval,"Business model (e.g. product-market fit, strat...",Noval,...,Noval,Noval,Noval,Noval,Concept,Prototype,Pilot,Noval,Noval,Noval
113,SF Export,X Prize Foundation,Amir,Banifatemi,Noval,Noval,Noval,Noval,"Business model (e.g. product-market fit, strat...",Financial (e.g. improving accounting practices...,...,Sub-Saharan Africa,East Asia and Pacific,South Asia,North America,Growth,Noval,Noval,Noval,Noval,Artificial Intelligence / Machine Learning; Bi...
114,SF Export,"X, The Moonshot Factory",Erica,Bliss,Noval,Noval,Noval,Noval,Noval,Noval,...,North America,Noval,Noval,Noval,Noval,Noval,Noval,Noval,Noval,Artificial Intelligence / Machine Learning; Bi...
